In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

<font size=18>Lesson 06 Homework</font>

# Textbook Problem 12.2-4

Reconsider the Wyndor Glass Co. problem presented in Sec. 3.1. Management now has decided that **only one** of the two new products should be produced, and the choice is to be made on the basis of maximizing profit. Introduce auxiliary binary variables to formulate an MIP model for this new version of the problem.

(a) Introduce auxiliary binary variables to formulate a mixed BIP model for this problem. Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

Maximize $Z = 3 x_1 + 5 x_2$

Subject to:

$
\begin{array}{ccccc}
 x_1 &   &      & \leq & 4 \\
     &   & 2x_2 & \leq & 12 \\
 3x_1 & + & 2x_2 & \leq & 18 \\
 xb_1 & + & xb_2 & = & 1 \\
\end{array}
$

$x_1 \geq 0$, $x_2 \geq 0$

b) Use Pyomo to solve this model. You can use a concrete or an abstract formulation.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

In [17]:
# unfold to see Pyomo solution with a vector of decision variables
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name="Wyndor")

products = ['drs', 'wdw']
bigM = 100

bounds_dict = {'drs': (0, 4), 'wdw': (0, 6)}
def bounds_rule(model, product):
    return (bounds_dict[product])

model.x = Var(products, domain = Reals, bounds=bounds_rule)
model.xb = Var(products, domain = Boolean)

# Objective
model.profit = Objective(expr=3.0*model.x['drs'] + 5.0*model.x['wdw'],
                         sense=maximize)

# Constraints
model.Constraint0 = Constraint(expr=model.xb['drs'] + model.xb['wdw'] == 1)
model.Constraint1 = Constraint(expr=3.0 * model.x['drs'] + 2.0 * model.x['wdw'] <= 18)

model.Constraints = ConstraintList()

for pr in products:  # produce product only if product is chosen
    model.Constraints.add(model.x[pr] <= 100 * model.xb[pr])

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Profit = ",
      numbers.format_currency(model.profit(), 'USD', locale='en_US'))

print("\nWhich products and how many:")
for pr in products:
    if bool(model.xb[pr]()):
        print(f"Produce {round(model.x[pr]())} of {pr}")
    else:
        print(f"Do not produce {pr}")

Profit =  $30.00

Which products and how many:
Do not produce drs
Produce 6 of wdw


# Textbook Problem 12.3-1

The Research and Development Division of the Progressive Company has been developing four possible new product lines. Management must now make a decision as to which of these four products actually will be produced and at what levels. Therefore, an operations research study has been requested to find the most profitable product mix.

A substantial cost is associated with beginning the production of any product, as given in the first row of the following table. Management’s objective is to find the product mix that maximizes the total profit (total net revenue minus start-up costs).

<img src="images/screen-prob12_3-1.png" alt="Solution" width="350" height="150">

Let the continuous decision variables $x_1, x_2, x_3,$ and $x_4$ be the production levels of products 1, 2, 3, and 4, respectively. Management has imposed the following policy constraints on these variables:

1. No more than two of the products can be produced.

2. Either product 3 or 4 can be produced only if either product 1 or 2 is produced.

3. Either 

$$5x_1 + 3x_2 + 6x_3 + 4x_4 \leq  6,000 $$

$$\text{or}$$ 

$$4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6,000 $$.

(a) Introduce auxiliary binary variables to formulate a mixed BIP model for this problem.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

Maximize $Z = 70x_1 - 50000 + 60x_2 - 40000 + 90x_3 - 70000 + 80x_4 - 60000$

Subject to:

$
\begin{array}{l}
xb_1 + xb_2 + xb_3 + xb_4 \leq 2 \\
-xb_1 - xb_2 + xb_3 + xb_4 \leq 0 \\
5x_1 + 3x_2 + 6x_3 + 4x_4 \leq  6,000 \\
4x_1 + 6x_2 + 3x_3 + 5x_4 \leq 6,000 \\
\end{array}
$

(b) Use Pyomo to solve this model. Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [20]:
# unfold to see Pyomo solution with a vector of decision variables
from pyomo.environ import *

# Concrete Model
model = ConcreteModel(name="prob2")

# define vars
products = ['Product1', 'Product2', 'Product3','Product4']
unit_profit = dict(zip(products, [70,60,90,80]))
startup_cost = dict(zip(products, [50000, 40000, 70000, 60000]))

model.x = Var(products, domain=NonNegativeReals)
model.xb = Var(products, domain=Boolean)

bigM = 10000

# Objective
model.profit = Objective(expr=sum((unit_profit[pr]*model.x[pr])-startup_cost[pr] for pr in products),
                         sense=maximize)

# Constraints
model.constraints = ConstraintList()

# no more than 2 products to be produced
model.constraints.add(sum(model.xb[product] for product in products) <= 2)

# Either product 3 or 4 can be produced only if either product 1 or 2 is produced
model.constraints.add(expr=-model.xb['Product1'] - model.xb['Product2'] + model.xb['Product3'] + model.xb['Product4'] <= 0)

# either formula
for pr in products: 
    model.constraints.add(5*model.x['Product1'] + 3*model.x['Product2'] + 6*model.x['Product3'] + 4*model.x['Product4'] <= 6000 + bigM * model.xb[pr])
    model.constraints.add(4*model.x['Product1'] + 6*model.x['Product2'] + 3*model.x['Product3'] + 5*model.x['Product4'] <= 6000 + bigM * (1-model.xb[pr])) 

for pr in products:  # produce product only if product is chosen
    model.constraints.add(model.x[pr] <= bigM * model.xb[pr])
    
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
print("\nWhich products and how many:")
for pr in products:
    if bool(model.xb[pr]()):
        print(f"Produce {round(model.x[pr]())} of {pr}")
    else:
        print(f"Do not produce {pr}")


Which products and how many:
Produce 667 of Product1
Do not produce Product2
Do not produce Product3
Produce 667 of Product4


# Textbook Problem 12.4-6

Speedy Delivery provides two-day delivery service of large parcels across the United States. Each morning at each collection center, the parcels that have arrived overnight are loaded onto several trucks for delivery throughout the area. Since the competitive battlefield in this business is speed of delivery, the parcels are divided among the trucks according to their geographical destinations to minimize the average time needed to make the deliveries.

On this particular morning, the dispatcher for the Blue River Valley Collection Center, Sharon Lofton, is hard at work. Her three drivers will be arriving in less than an hour to make the day’s deliveries. There are nine parcels to be delivered, all at locations many miles apart. As usual, Sharon has loaded these locations into her computer. She is using her company’s special software package, a decision support system called Dispatcher. The first thing Dispatcher does is use these locations to generate a considerable number of attractive possible routes for the individual delivery trucks. These routes are shown in the following table (where the numbers in each column indicate the order of the deliveries), along with the estimated time required to traverse the route.

<img src="images/screen-prob12_4-6.png" alt="Solution" width="350" height="150">


Dispatcher is an interactive system that shows these routes to Sharon for her approval or modification. (For example, the computer may not know that flooding has made a particular route infeasible.) After Sharon approves these routes as attractive possibilities with reasonable time estimates, Dispatcher next formulates and solves a BIP model for selecting three routes that minimize their total time while including each delivery location on exactly one route. This morning, Sharon does approve all the routes.

(a) Formulate this BIP model.  Include a picture or LaTeX of the mathematical formulation in the next cell.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

$
x = routes\\
y = delivery\ locations\\
$

Minimize $Z = 6x_0 + 4x_1 + 7x_2 + 5x_3 + 4x_4 + 6x_5 + 5x_6 + 3x_7 + 7x_8 + 6x_9$

Subject to:

$
\begin{array}{l}
x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 + x_9 = 3 \\
y_j = 1
\end{array}
$

(b) Use Pyomo solve this model.  Use an abstract formulation.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [23]:
# unfold to see Pyomo solution with a vector of decision variables
import numpy as np
from pyomo.environ import *

# define vars
routes = ['route1', 'route2', 'route3','route4', 'route5', 'route6','route7', 'route8','route9', 'route10']
locations = ['A', 'B', 'C','D', 'E', 'F','G', 'H','I']
route_time = dict(zip(routes, [6,4,7,5,4,6,5,3,7,6]))
routes_locations = [[1,0,0,1,0,0,1,0,0],
                    [0,1,0,0,0,1,0,0,1],
                    [0,0,1,0,1,0,0,1,0],
                    [0,1,1,0,1,0,0,0,1],
                    [1,0,0,0,0,1,0,1,0],
                    [0,1,0,1,1,0,0,0,0],
                    [0,0,1,0,0,0,1,0,1],
                    [0,0,0,1,0,0,1,0,0],
                    [1,1,1,0,0,0,0,0,0],
                    [0,1,0,0,0,0,1,1,0]]

# get tranpose of routes_locations
locations_routes = np.array(routes_locations).T.tolist()

routes_locations_mapped = {}
locations_routes_mapped = {}

# create mapping dicts
for i, (route, route_locations) in enumerate(zip(routes,routes_locations)):
    routes_locations_mapped[route] = dict(zip(locations, route_locations))
    
for i, (loc, location_route) in enumerate(zip(locations,locations_routes)):
    locations_routes_mapped[loc] = dict(zip(routes, location_route))

# Concrete Model
model = ConcreteModel(name="prob3")

model.routes = Var(routes, domain=Boolean)
model.routes_locations = Var(routes, locations, domain=Boolean)
model.locations = Var(locations, domain=Boolean)

# Objective
model.time = Objective(expr=sum(route_time[route]*model.routes[route] for route in routes),
                         sense=minimize)

# Constraints
model.Constraints = ConstraintList()

# total routes == 3
model.Constraints.add(sum(model.routes[route] for route in routes) == 3)

# every delivery should be made between 3 routes
for loc in locations:
    model.Constraints.add(
        sum(model.routes[route] * locations_routes_mapped[loc][route] for route in routes) >= 1)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# display solution
import babel.numbers as numbers  # needed to display as currency
print("Minimum time = ", model.time())
print("\nWhich routes:")
for route in routes:
    if bool(model.routes[route]()):
        print(f"{route}")

Minimum time =  12.0

Which routes:
route4
route5
route8


# Knapsack optimization

We'll revisit the Knapsack Optimization problem from Lesson 4 and 5. This time, we'll solve it using integer programming.

Given a set of items, each with a weight and a value, use binary variables and Pyomo to determine which items to include in a collection such that the total weight is less than or equal to a given limit and the total value is as large as possible. We will start with 20 items and you need to determine the collection of items that maximizes the value and keeps the total weight less than or equal to 50. 

Use the problem data as described below:

In [ ]:
#Problem Data - generate random weights and values for a knapsack problem
import numpy as np
num_items = 20
np.random.seed(seed=123)
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

Your Pyomo solution should go in the next cell.  Your code should still work if the number of items is changed to 40 or more.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [ ]:
from pyomo.environ import *